# Women Cloth Reviews Prediction with Multi Nomial Naïve Bayes
---
Multinomial Naïve Bayes algorithm is a probabilistic learning method that is mostly used in Natural Language Processing. It is also suitable for text classification with discrete feature. In this project it will be used to build a women cloth review prediction model.

Data Source: [YBIFoundation/ProjectHub-MachineLearning Women Clothing Commerce Review dataset](https://raw.githubusercontent.com/YBIFoundation/ProjectHub-MachineLearning/main/Women%20Clothing%20E-Commerce%20Review.csv)

### Importing Libraries

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Importing Dataset

In [5]:
df = pd.read_csv("https://raw.githubusercontent.com/YBIFoundation/ProjectHub-MachineLearning/main/Women%20Clothing%20E-Commerce%20Review.csv")

### Data visualization

Samples of the data is visualized to better understand how it is structured.

In [6]:
df.head()

,Clothing ID,Age,Title,Review,Rating,Recommended,Positive Feedback,Division,Department,Category
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Clothing ID        23486 non-null  int64 
 1   Age                23486 non-null  int64 
 2   Title              19676 non-null  object
 3   Review             22641 non-null  object
 4   Rating             23486 non-null  int64 
 5   Recommended        23486 non-null  int64 
 6   Positive Feedback  23486 non-null  int64 
 7   Division           23472 non-null  object
 8   Department         23472 non-null  object
 9   Category           23472 non-null  object
dtypes: int64(5), object(5)
memory usage: 1.8+ MB


In [8]:
df.shape

(23486, 10)

### Data preprocessing

Calling `isna()` method along with the `sum()` method on dataframe `df` to find the Review columns with no review text for further processing.

In [9]:
df.isna().sum()

Clothing ID             0
Age                     0
Title                3810
Review                845
Rating                  0
Recommended             0
Positive Feedback       0
Division               14
Department             14
Category               14
dtype: int64

Filling missing values in the Review column with the value `No review is given`.

In [12]:
df.loc[df['Review'] == ""] = np.nan  # Use .loc[] to avoid chained assignment
df['Review'] = df['Review'].fillna("No review is given")  # Avoid inplace=True


In [13]:
df['Review']

0        Absolutely wonderful - silky and sexy and comf...
1        Love this dress!  it's sooo pretty.  i happene...
2        I had such high hopes for this dress and reall...
3        I love, love, love this jumpsuit. it's fun, fl...
4        This shirt is very flattering to all due to th...
                               ...                        
23481    I was very happy to snag this dress at such a ...
23482    It reminds me of maternity clothes. soft, stre...
23483    This fit well, but the top was very see throug...
23484    I bought this dress for a wedding i have this ...
23485    This dress in a lovely platinum is feminine an...
Name: Review, Length: 23486, dtype: object

### Defining Target Variable (y) and Feature Variables (X)

In [14]:
df.columns

Index(['Clothing ID', 'Age', 'Title', 'Review', 'Rating', 'Recommended',
       'Positive Feedback', 'Division', 'Department', 'Category'],
      dtype='object')

In [15]:
x = df['Review']
y = df['Rating']
df['Rating'].value_counts()

Rating
5    13131
4     5077
3     2871
2     1565
1      842
Name: count, dtype: int64

### Train Test Split

In [20]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, stratify=y, random_state=2529)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((16440,), (7046,), (16440,), (7046,))

### Getting Tokens from Feature Text

Using `CountVectorizer` from sci-kit learn

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(lowercase=True, analyzer='word', ngram_range=(2, 3), stop_words='english', max_features=50000)
x_train = cv.fit_transform(x_train)
cv.get_feature_names_out()

array(['00 big', '00 fits', '00 petite', ..., 'zipper zip',
       'zippered pockets', 'zippers buttons'],
      shape=(50000,), dtype=object)

In [22]:
x_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(16440, 50000))

In [23]:
x_test = cv.fit_transform(x_test)
cv.get_feature_names_out()

array(['00 24 xs', '00 aren', '00 aren size', ..., 'zone roomy looking',
       'zone style', 'zone style daughter'], shape=(50000,), dtype=object)

In [25]:
 # Check if x_test is sparse
from scipy.sparse import issparse

if issparse(x_test):
    print("x_test is a sparse matrix, no need to convert to array.")


x_test is a sparse matrix, no need to convert to array.


### Model training

Using Multinomial Naïve Bayes algorithm, which is implemented in sci-kit as `MultinomialNB`

In [26]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(x_train, y_train)

MultinomialNB()

### Model prediction

In [27]:
y_pred = model.predict(x_test)
y_pred.shape

(7046,)

In [28]:
y_pred

array([1, 1, 5, ..., 5, 2, 3], shape=(7046,))

Getting probability of each predicted class

In [29]:
model.predict_proba(x_test)

array([[7.82182384e-01, 1.63488184e-01, 3.74139474e-02, 1.65518087e-02,
        3.63676438e-04],
       [3.74659919e-01, 6.94406435e-02, 7.87883362e-02, 3.72192784e-01,
        1.04918318e-01],
       [4.61718254e-02, 1.47323604e-01, 1.09233446e-01, 2.88690279e-01,
        4.08580845e-01],
       ...,
       [6.67388724e-02, 1.98214997e-01, 1.51704283e-01, 2.80428047e-01,
        3.02913801e-01],
       [7.37276589e-03, 9.91459241e-01, 1.14358937e-03, 2.41622343e-05,
        2.41460534e-07],
       [2.07140578e-01, 8.42145166e-02, 6.43688717e-01, 6.47944624e-02,
        1.61725966e-04]], shape=(7046, 5))

### Model evaluation

In [30]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_test, y_pred))

[[  62   44   53   53   41]
 [ 127   86   72   89   96]
 [ 276  178  124  117  166]
 [ 537  301  207  191  287]
 [1281  785  554  544  775]]


In [31]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.03      0.25      0.05       253
           2       0.06      0.18      0.09       470
           3       0.12      0.14      0.13       861
           4       0.19      0.13      0.15      1523
           5       0.57      0.20      0.29      3939

    accuracy                           0.18      7046
   macro avg       0.19      0.18      0.14      7046
weighted avg       0.38      0.18      0.22      7046



### Recategorizing ratings as Poor (0) and Good (1)

In [32]:
df["Rating"].value_counts()

Rating
5    13131
4     5077
3     2871
2     1565
1      842
Name: count, dtype: int64

re-rating 1,2,3 as 0 and 4,5 as 1

In [33]:
df.replace({'Rating': { 1:0, 2:0, 3:0, 4:1, 5:1 }}, inplace=True)
y = df['Rating']
x = df['Review']

### Train Test Split

In [34]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, stratify=y, random_state=2529)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((16440,), (7046,), (16440,), (7046,))

### Getting Tokens from Feature Text

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(lowercase=True, analyzer='word', ngram_range=(2, 3), stop_words='english', max_features=50000)
x_train = cv.fit_transform(x_train)
x_test = cv.fit_transform(x_test)

### Model re-training

In [36]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(x_train, y_train)

MultinomialNB()

### Model prediction

In [37]:
y_pred = model.predict(x_test)
y_pred.shape

(7046,)

In [38]:
y_pred

array([0, 1, 1, ..., 1, 1, 0], shape=(7046,))

### Model evaluation

In [39]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_test, y_pred))

[[ 785  798]
 [2873 2590]]


In [40]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.21      0.50      0.30      1583
           1       0.76      0.47      0.59      5463

    accuracy                           0.48      7046
   macro avg       0.49      0.48      0.44      7046
weighted avg       0.64      0.48      0.52      7046



### Explanation

This project is focused on building a prediction model. At first, the all required libraries and a test dataset are imported. The dataset was evaluated and pre processed to prepare for it for processing, then a portion of it was kept for testing and the rest was used to train the model. The model was used to get some prediction dataset. Finnaly, prediction accuracy was checked against the test dataset, some adjusment were made and the model was re-trained for better accuracy.